# RNN

In [6]:
from keras.utils import pad_sequences
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer #Data transformation
from sklearn.linear_model import LogisticRegression #Prediction Model
from sklearn.metrics import accuracy_score #Comparison between real and predicted
from sklearn.model_selection import train_test_split #Data testing
from tensorflow.keras import Input, Model
from tensorflow.keras import models, layers, optimizers, losses, metrics
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, Embedding, Flatten
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
import json
import matplotlib.pyplot as plt #Plotting properties
#import nltk
import numpy as np
import os
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report
from pprint import pprint
import tensorflow as tf

In [7]:
train_data = pd.read_csv('./preprocessing_train.csv', index_col=0)
val_data = pd.read_csv('./preprocessing_val.csv', index_col=0)

In [19]:
x,y= [], []
x = train_data['lower'].values.tolist()
y = train_data['type'].values.tolist()

x2,y2= [], []
x2 = val_data['lower'].values.tolist()
y2 = val_data['type'].values.tolist()

In [12]:
test_percent = 0.2 # test data percent
# make train and test data usin train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_percent)

In [17]:
len(x_train) # 31891
len(x_test) # 7973
len(x) # 39864
len(y) # 39864

39864

In [49]:
def build_model(train_data): # to make rnn model
    train_data = tf.data.Dataset.from_tensor_slices(train_data)
    model = Sequential()
    model.add(Input(shape=(1,), dtype="string")) # input one string data (comment)
    max_tokens = 15000 # dictionary size
    max_len = 50 # comment to vectorize size
    vectorize_layer = TextVectorization( # make textvectorization 
      max_tokens=max_tokens,
      output_mode="int",
      output_sequence_length=max_len,
    )
    
    vectorize_layer.adapt(train_data.batch(64))
    model.add(vectorize_layer)
    model.add(layers.Embedding(max_tokens + 1,output_dim= 200))
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(1, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation="sigmoid"))
    return model

In [50]:
rnn_model =build_model(x_train)
rnn_model.compile( # rnn model compile
        optimizer=  "adam",
        loss='binary_crossentropy',
        metrics=['accuracy'])
rnn_model.save("RNN_model")

NameError: name 'Dropout' is not defined

In [51]:
# model training
history = rnn_model.fit(x_train, y_train,
                   epochs = 5,  batch_size = 128  , validation_data = (x2, y2) )
rnn_model.summary()

Epoch 1/5
250/250 [==============================] - 19s 75ms/step - loss: 0.0068 - accuracy: 0.9974 - val_loss: 0.0448 - val_accuracy: 0.9834
Epoch 2/5
250/250 [==============================] - 18s 73ms/step - loss: 0.0069 - accuracy: 0.9970 - val_loss: 0.0404 - val_accuracy: 0.9871
Epoch 3/5
250/250 [==============================] - 17s 68ms/step - loss: 0.0066 - accuracy: 0.9974 - val_loss: 0.0456 - val_accuracy: 0.9834
Epoch 4/5
250/250 [==============================] - 18s 70ms/step - loss: 0.0065 - accuracy: 0.9970 - val_loss: 0.0471 - val_accuracy: 0.9852
Epoch 5/5
250/250 [==============================] - 21s 84ms/step - loss: 0.0067 - accuracy: 0.9974 - val_loss: 0.0469 - val_accuracy: 0.9871
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_4 (TextV  (None, 50)               0         
 ectorization)                                                   

In [40]:
rnn_model = tf.keras.models.load_model("./RNN_model", custom_objects={"TextVectorization":TextVectorization})
